In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [26]:
# data processing

df= pd.read_csv("movie_metadata.csv")
#drop useless(we think it's) columns
df=df.drop(columns=["color","director_name","actor_1_name","actor_2_name","actor_3_name","language","country","movie_imdb_link","plot_keywords","movie_title","title_year","cast_total_facebook_likes","facenumber_in_poster","aspect_ratio","movie_facebook_likes"])
#drop row with missing(NAN)value
df=df.dropna()
# count how many movies (row )left
df.count()

#encode genres(multi-label)
df['genres']=df['genres'].str.split('|')
mlb= MultiLabelBinarizer()
genre_encoded=pd.DataFrame(mlb.fit_transform(df['genres']), columns=mlb.classes_, index=df.index)
df=df.drop(columns=["genres"]).join(genre_encoded)

# encode content_rating
le=LabelEncoder()
df['content_rating'] = le.fit_transform(df['content_rating'])

#the value we want to predict
scores=df["imdb_score"]
df= df.drop(columns=["imdb_score"])


In [27]:
# pre processing
X_train,X_test,y_train,y_test=train_test_split(df,scores,test_size=0.2,shuffle=True,random_state=42)
# train model

model=RandomForestRegressor()
model.fit(X_train,y_train)
# predict
y_pred=model.predict(X_test)
# evaluate
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")


Mean Squared Error: 0.4407266179921775
R^2 Score: 0.6077843452726306
